In [1]:
import sys
import os
from pathlib import Path

IN_COLAB = False

# if in colab, make colab setup
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    IN_COLAB = True
    ZIP_PATH = "/content/drive/MyDrive/embedded_ml_data/VOCdevkit.zip"
    FILENAME = "dev-notebook.ipynb"
    BRANCH = "michael"
    !git clone -b $BRANCH https://github.com/yannickfunk/EmbeddedMLLab tmp
    !rm tmp/$FILENAME
    !mv tmp/* .
    !rm -rf tmp
    !rm -rf sample_data
    !pip install -r requirements.txt

    # setup data
    !cp  $ZIP_PATH data/
    %pushd data
    !unzip -qq VOCdevkit.zip
    %popd

DATA_PATH = "data/"

# sys.path.append('../')

Prepare Tensorboard logger

In [3]:
import pytorch_lightning as pl

if IN_COLAB == True:
    %load_ext tensorboard
    %tensorboard --logdir $LOGDIR
   
tensorboard = pl.loggers.TensorBoardLogger(save_dir="")


Prepare model

In [7]:
from models.tinyyolov2 import TinyYoloV2PersonOnly
import pytorch_lightning as pl
from utils.dataloader import VOCDataModule

trainer = pl.Trainer(max_epochs=30, auto_scale_batch_size='binsearch',accelerator='auto', logger=tensorboard, fast_dev_run=not IN_COLAB)


model = TinyYoloV2PersonOnly()
model.load_pt_from_disk(DATA_PATH + '/voc_pretrained.pt')

data= VOCDataModule(person_only=True)




GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


Tune and train model

In [8]:
trainer.tune(model, data)
trainer.fit(model, data)

c:\Users\Michael\miniconda3\envs\embedded-ml\lib\site-packages\pytorch_lightning\tuner\batch_size_scaling.py:38: UserWarning: Skipping batch size scaler since `fast_dev_run` is enabled.
  rank_zero_warn("Skipping batch size scaler since `fast_dev_run` is enabled.")

   | Name  | Type            | Params
-------------------------------------------
0  | loss  | YoloLoss        | 0     
1  | pad   | ReflectionPad2d | 0     
2  | conv1 | Conv2d          | 432   
3  | bn1   | BatchNorm2d     | 32    
4  | conv2 | Conv2d          | 4.6 K 
5  | bn2   | BatchNorm2d     | 64    
6  | conv3 | Conv2d          | 18.4 K
7  | bn3   | BatchNorm2d     | 128   
8  | conv4 | Conv2d          | 73.7 K
9  | bn4   | BatchNorm2d     | 256   
10 | conv5 | Conv2d          | 294 K 
11 | bn5   | BatchNorm2d     | 512   
12 | conv6 | Conv2d          | 1.2 M 
13 | bn6   | BatchNorm2d     | 1.0 K 
14 | conv7 | Conv2d          | 4.7 M 
15 | bn7   | BatchNorm2d     | 2.0 K 
16 | conv8 | Conv2d          | 9.4 M 
17 | 

Freezing layers: conv1.weight, bn1.weight, bn1.bias, conv2.weight, bn2.weight, bn2.bias, conv3.weight, bn3.weight, bn3.bias, conv4.weight, bn4.weight, bn4.bias, conv5.weight, bn5.weight, bn5.bias, conv6.weight, bn6.weight, bn6.bias, conv7.weight, bn7.weight, bn7.bias, 
Freezing layers: conv1.weight, bn1.weight, bn1.bias, conv2.weight, bn2.weight, bn2.bias, conv3.weight, bn3.weight, bn3.bias, conv4.weight, bn4.weight, bn4.bias, conv5.weight, bn5.weight, bn5.bias, conv6.weight, bn6.weight, bn6.bias, conv7.weight, bn7.weight, bn7.bias, 


c:\Users\Michael\miniconda3\envs\embedded-ml\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\Michael\miniconda3\envs\embedded-ml\lib\site-packages\pytorch_lightning\trainer\trainer.py:1558: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(
c:\Users\Michael\miniconda3\envs\embedded-ml\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling of

Training: 0it [00:00, ?it/s]

c:\Users\Michael\miniconda3\envs\embedded-ml\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.
